### MLP1L with different scaler (MLP1 generated Samples)

In [13]:
import os
import cv2
import numpy as np
import torch
import pandas as pd
import pyperclip
from pathlib import Path
from io import StringIO
from sklearn.preprocessing import StandardScaler

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model = torch.jit.load(r'Models and Data splits\MainModel_MLPSca.pt')
model.to(device)
model.eval()

# Root directory for your adversarial images
adversarial_root = Path(r"Generated Data\ZC-CSA_images\Adversarial")
all_image_paths = list(adversarial_root.rglob("*.png"))

# Step 1: Load everything into memory (vectors + metadata)
vectors   = []
metadata  = []

for file_path in all_image_paths:
    image = cv2.imread(str(file_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Warning: Could not load image {file_path}")
        continue

    # Parse context folder
    context_folder = file_path.parts[-3] if len(file_path.parts) >= 3 else "UnknownFolder"

    # Parse filename: e.g. '495_t9_p2_m814.39.png'
    fname  = file_path.stem
    parts  = fname.split("_")
    idx    = int(parts[0])
    true_l = int(parts[1].lstrip("t"))
    pred_l = int(parts[2].lstrip("p"))
    mag    = float(parts[3].lstrip("m"))

    flat = image.astype(np.float32).flatten()
    vectors.append(flat)
    metadata.append({
        "folder":   context_folder,
        "Index":    idx,
        "True":     true_l,
        "Adversarial": pred_l,
        "Magnitude":   mag
    })

if not vectors:
    raise RuntimeError("No images found or loaded.")

# Step 2: Fit StandardScaler on the full dataset
X = np.vstack(vectors)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Convert to torch Tensor and run inference in one batch
inputs = torch.tensor(X_scaled, dtype=torch.float32, device=device)
with torch.no_grad():
    outputs = model(inputs)
preds = torch.argmax(outputs, dim=1).cpu().numpy()

# Step 4: Aggregate results per folder
results_by_folder = {}
for meta, pred in zip(metadata, preds):
    folder = meta.pop("folder")
    record = dict(meta, ModelPred=int(pred))
    results_by_folder.setdefault(folder, []).append(record)

# Step 5: Print tables and copy to clipboard
output = StringIO()
for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    header    = f"\n=== Results from Folder: {folder} ===\n"
    table_str = df.to_string(index=False)
    print(header + table_str)
    output.write(header)
    output.write(table_str + "\n")

pyperclip.copy(output.getvalue())
print("\n✅ All formatted tables have been copied to the clipboard!")


# --- Misclassification Summary ---
# Per‐folder
print("\n=== Misclassification Summary by Folder ===")
total_all = 0
mismatch_all = 0

for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    total    = len(df)
    mismatches = (df["True"] != df["ModelPred"]).sum()
    pct      = mismatches / total * 100
    print(f"{folder:15s}  {mismatches:4d}/{total:4d} misclassified   ({pct:5.2f}%)")
    total_all    += total
    mismatch_all += mismatches

# Overall
pct_all = mismatch_all / total_all * 100 if total_all else 0.0
print("\nOverall misclassification rate:")
print(f"    {mismatch_all}/{total_all} samples   ({pct_all:.2f}%)")




=== Results from Folder: ZC-CSA_images ===
 Index  True  Adversarial  Magnitude  ModelPred
   102     2            7    1150.38          7
   103     2            8    1133.78          2
   104     2            7     852.70          7
   107     2            7    1087.49          7
   111     2            7    1155.91          7
   114     2            7    1132.34          2
   116     2            5    1152.05          2
   118     2            7    1108.96          7
    11     0            2    1138.52          0
   120     2            7    1183.65          2
   121     2            0    1138.58          0
   123     2            8    1111.15          9
   128     2            7    1124.17          2
   139     2            3    1168.42          3
    13     0            6    1126.73          6
   141     2            6    1155.44          2
   144     2            3     934.63          2
   150     3            5    1166.40          3
   153     3            5    1115.46        

### MLP1L with different scaler (CNN generated Samples)

In [15]:
import os
import cv2
import numpy as np
import torch
import pandas as pd
import pyperclip
from pathlib import Path
from io import StringIO
from sklearn.preprocessing import StandardScaler

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model = torch.jit.load(r'Models and Data splits\MainModel_MLPSca.pt')
model.to(device)
model.eval()

# Root directory for your adversarial images
adversarial_root = Path(r"Generated Data\cnn_ZC-CSA_images\Adversarial")
all_image_paths = list(adversarial_root.rglob("*.png"))

# Step 1: Load everything into memory (vectors + metadata)
vectors   = []
metadata  = []

for file_path in all_image_paths:
    image = cv2.imread(str(file_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Warning: Could not load image {file_path}")
        continue

    # Parse context folder
    context_folder = file_path.parts[-3] if len(file_path.parts) >= 3 else "UnknownFolder"

    # Parse filename: e.g. '495_t9_p2_m814.39.png'
    fname  = file_path.stem
    parts  = fname.split("_")
    idx    = int(parts[0])
    true_l = int(parts[1].lstrip("t"))
    pred_l = int(parts[2].lstrip("p"))
    mag    = float(parts[3].lstrip("m"))

    flat = image.astype(np.float32).flatten()
    vectors.append(flat)
    metadata.append({
        "folder":   context_folder,
        "Index":    idx,
        "True":     true_l,
        "Adversarial": pred_l,
        "Magnitude":   mag
    })

if not vectors:
    raise RuntimeError("No images found or loaded.")

# Step 2: Fit StandardScaler on the full dataset
X = np.vstack(vectors)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Convert to torch Tensor and run inference in one batch
inputs = torch.tensor(X_scaled, dtype=torch.float32, device=device)
with torch.no_grad():
    outputs = model(inputs)
preds = torch.argmax(outputs, dim=1).cpu().numpy()

# Step 4: Aggregate results per folder
results_by_folder = {}
for meta, pred in zip(metadata, preds):
    folder = meta.pop("folder")
    record = dict(meta, ModelPred=int(pred))
    results_by_folder.setdefault(folder, []).append(record)

# Step 5: Print tables and copy to clipboard
output = StringIO()
for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    header    = f"\n=== Results from Folder: {folder} ===\n"
    table_str = df.to_string(index=False)
    print(header + table_str)
    output.write(header)
    output.write(table_str + "\n")

pyperclip.copy(output.getvalue())
print("\n✅ All formatted tables have been copied to the clipboard!")


# --- Misclassification Summary ---
# Per‐folder
print("\n=== Misclassification Summary by Folder ===")
total_all = 0
mismatch_all = 0

for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    total    = len(df)
    mismatches = (df["True"] != df["ModelPred"]).sum()
    pct      = mismatches / total * 100
    print(f"{folder:15s}  {mismatches:4d}/{total:4d} misclassified   ({pct:5.2f}%)")
    total_all    += total
    mismatch_all += mismatches

# Overall
pct_all = mismatch_all / total_all * 100 if total_all else 0.0
print("\nOverall misclassification rate:")
print(f"    {mismatch_all}/{total_all} samples   ({pct_all:.2f}%)")




=== Results from Folder: cnn_ZC-CSA_images ===
 Index  True  Adversarial  Magnitude  ModelPred
     0     0            8    1109.47          0
   104     2            1    1157.97          2
   106     2            8    1149.38          2
   107     2            7    1160.00          2
   117     2            7    1141.05          2
   118     2            3    1166.82          3
   119     2            8    1176.49          2
   120     2            3    1159.97          2
   121     2            3    1123.15          2
   124     2            1    1099.96          2
   129     2            1    1141.25          2
   130     2            1    1154.87          2
   135     2            7    1166.69          2
   139     2            8    1157.05          2
   141     2            3    1129.80          3
   145     2            3    1149.30          2
   154     3            9    1170.49          7
   159     3            5    1127.94          3
   160     3            9    1130.50    

### CNN with different scaler (MLP1 generated Samples)

In [17]:
import cv2
import numpy as np
import torch
import pandas as pd
import pyperclip
from sklearn.preprocessing import StandardScaler
from pathlib import Path
from io import StringIO

# 1. Setup device & load your saved CNN
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.jit.load(r"Models and Data splits\CNNScl.pt").to(device)
model.eval()

# 2. Locate all adversarial PNGs
adversarial_root = Path(r"Generated Data\ZC-CSA_images\Adversarial")
all_image_paths  = list(adversarial_root.rglob("*.png"))

# 3. Fit a StandardScaler on *all* raw‑pixel values (0‑255)
print("Fitting StandardScaler on dataset …")
pixel_bank = []
for fp in all_image_paths:
    img = cv2.imread(str(fp), cv2.IMREAD_GRAYSCALE)
    if img is not None:
        pixel_bank.append(img.astype(np.float32).reshape(-1, 1))  # keep 0‑255 range
scaler = StandardScaler().fit(np.vstack(pixel_bank))
mu    = scaler.mean_[0]            # or  scaler.mean_.item()
sigma = np.sqrt(scaler.var_[0])    # or  np.sqrt(scaler.var_).item()
print(f"Scaler fitted.  μ ≈ {mu:.2f}, σ ≈ {sigma:.2f}")


results_by_folder = {}

# 4. Inference loop
for file_path in all_image_paths:
    try:
        # Load grayscale image
        img = cv2.imread(str(file_path), cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Warning: Could not load image {file_path}")
            continue

        # --- StandardScaler normalisation & reshape to (1,1,H,W) ---
        H, W      = img.shape
        flat      = img.astype(np.float32).reshape(-1, 1)     # 0‑255 → float32
        flat_norm = scaler.transform(flat).astype(np.float32) # standardise
        img_norm  = flat_norm.reshape(H, W)

        tensor = torch.from_numpy(img_norm).unsqueeze(0).unsqueeze(0).to(device)
        
        # Forward pass
        with torch.no_grad():
            outputs = model(tensor)
        predicted = outputs.argmax(dim=1).item()

        # Determine folder context
        context_folder = file_path.parts[-3] if len(file_path.parts) >= 3 else "UnknownFolder"

        # Parse filename: e.g. "495_t9_p2_m814.39.png"
        fname = file_path.stem
        idx, t_lbl, p_lbl, m_val = fname.split("_")
        index      = int(idx)
        true_label = int(t_lbl.lstrip("t"))
        pred_label = int(p_lbl.lstrip("p"))
        magnitude  = float(m_val.lstrip("m"))

        # Accumulate results
        results_by_folder.setdefault(context_folder, []).append({
            "Index":       index,
            "True":        true_label,
            "Adversarial": pred_label,
            "Magnitude":   magnitude,
            "ModelPred":   predicted
        })

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# 5. Print tables & copy to clipboard
output = StringIO()
for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    header    = f"\n=== Results from Folder: {folder} ===\n"
    table_str = df.to_string(index=False)
    print(header + table_str)
    output.write(header)
    output.write(table_str + "\n")

pyperclip.copy(output.getvalue())
print("\n✅ All formatted tables have been copied to the clipboard!")

# --- Misclassification Summary ---
print("\n=== Misclassification Summary by Folder ===")
total_all     = 0
mismatch_all  = 0

for folder, records in results_by_folder.items():
    df         = pd.DataFrame(records)
    total      = len(df)
    mismatches = (df["True"] != df["ModelPred"]).sum()
    pct        = mismatches / total * 100
    print(f"{folder:15s}  {mismatches:4d}/{total:4d} misclassified   ({pct:5.2f}%)")
    total_all    += total
    mismatch_all += mismatches

pct_all = mismatch_all / total_all * 100 if total_all else 0.0
print("\nOverall misclassification rate:")
print(f"    {mismatch_all}/{total_all} samples   ({pct_all:.2f}%)")


Fitting StandardScaler on dataset …
Scaler fitted.  μ ≈ 38.03, σ ≈ 66.71

=== Results from Folder: ZC-CSA_images ===
 Index  True  Adversarial  Magnitude  ModelPred
   102     2            7    1150.38          2
   103     2            8    1133.78          8
   104     2            7     852.70          2
   107     2            7    1087.49          2
   111     2            7    1155.91          2
   114     2            7    1132.34          2
   116     2            5    1152.05          2
   118     2            7    1108.96          2
    11     0            2    1138.52          0
   120     2            7    1183.65          2
   121     2            0    1138.58          2
   123     2            8    1111.15          8
   128     2            7    1124.17          2
   139     2            3    1168.42          2
    13     0            6    1126.73          0
   141     2            6    1155.44          2
   144     2            3     934.63          8
   150     3       

### CNN with different scaler (CNN generated Samples)

In [19]:
import cv2
import numpy as np
import torch
import pandas as pd
import pyperclip
from sklearn.preprocessing import StandardScaler
from pathlib import Path
from io import StringIO

# 1. Setup device & load your saved CNN
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.jit.load(r"Models and Data splits\CNNScl.pt").to(device)
model.eval()

# 2. Locate all adversarial PNGs
adversarial_root = Path(r"Generated Data\cnn_ZC-CSA_images\Adversarial")
all_image_paths  = list(adversarial_root.rglob("*.png"))

# 3. Fit a StandardScaler on *all* raw‑pixel values (0‑255)
print("Fitting StandardScaler on dataset …")
pixel_bank = []
for fp in all_image_paths:
    img = cv2.imread(str(fp), cv2.IMREAD_GRAYSCALE)
    if img is not None:
        pixel_bank.append(img.astype(np.float32).reshape(-1, 1))  # keep 0‑255 range
scaler = StandardScaler().fit(np.vstack(pixel_bank))
mu    = scaler.mean_[0]            # or  scaler.mean_.item()
sigma = np.sqrt(scaler.var_[0])    # or  np.sqrt(scaler.var_).item()
print(f"Scaler fitted.  μ ≈ {mu:.2f}, σ ≈ {sigma:.2f}")


results_by_folder = {}

# 4. Inference loop
for file_path in all_image_paths:
    try:
        # Load grayscale image
        img = cv2.imread(str(file_path), cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Warning: Could not load image {file_path}")
            continue

        # --- StandardScaler normalisation & reshape to (1,1,H,W) ---
        H, W      = img.shape
        flat      = img.astype(np.float32).reshape(-1, 1)     # 0‑255 → float32
        flat_norm = scaler.transform(flat).astype(np.float32) # standardise
        img_norm  = flat_norm.reshape(H, W)

        tensor = torch.from_numpy(img_norm).unsqueeze(0).unsqueeze(0).to(device)
        
        # Forward pass
        with torch.no_grad():
            outputs = model(tensor)
        predicted = outputs.argmax(dim=1).item()

        # Determine folder context
        context_folder = file_path.parts[-3] if len(file_path.parts) >= 3 else "UnknownFolder"

        # Parse filename: e.g. "495_t9_p2_m814.39.png"
        fname = file_path.stem
        idx, t_lbl, p_lbl, m_val = fname.split("_")
        index      = int(idx)
        true_label = int(t_lbl.lstrip("t"))
        pred_label = int(p_lbl.lstrip("p"))
        magnitude  = float(m_val.lstrip("m"))

        # Accumulate results
        results_by_folder.setdefault(context_folder, []).append({
            "Index":       index,
            "True":        true_label,
            "Adversarial": pred_label,
            "Magnitude":   magnitude,
            "ModelPred":   predicted
        })

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# 5. Print tables & copy to clipboard
output = StringIO()
for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    header    = f"\n=== Results from Folder: {folder} ===\n"
    table_str = df.to_string(index=False)
    print(header + table_str)
    output.write(header)
    output.write(table_str + "\n")

pyperclip.copy(output.getvalue())
print("\n✅ All formatted tables have been copied to the clipboard!")

# --- Misclassification Summary ---
print("\n=== Misclassification Summary by Folder ===")
total_all     = 0
mismatch_all  = 0

for folder, records in results_by_folder.items():
    df         = pd.DataFrame(records)
    total      = len(df)
    mismatches = (df["True"] != df["ModelPred"]).sum()
    pct        = mismatches / total * 100
    print(f"{folder:15s}  {mismatches:4d}/{total:4d} misclassified   ({pct:5.2f}%)")
    total_all    += total
    mismatch_all += mismatches

pct_all = mismatch_all / total_all * 100 if total_all else 0.0
print("\nOverall misclassification rate:")
print(f"    {mismatch_all}/{total_all} samples   ({pct_all:.2f}%)")


Fitting StandardScaler on dataset …
Scaler fitted.  μ ≈ 39.83, σ ≈ 69.65

=== Results from Folder: cnn_ZC-CSA_images ===
 Index  True  Adversarial  Magnitude  ModelPred
     0     0            8    1109.47          8
   104     2            1    1157.97          8
   106     2            8    1149.38          2
   107     2            7    1160.00          2
   117     2            7    1141.05          2
   118     2            3    1166.82          3
   119     2            8    1176.49          8
   120     2            3    1159.97          1
   121     2            3    1123.15          2
   124     2            1    1099.96          2
   129     2            1    1141.25          2
   130     2            1    1154.87          2
   135     2            7    1166.69          2
   139     2            8    1157.05          8
   141     2            3    1129.80          3
   145     2            3    1149.30          8
   154     3            9    1170.49          8
   159     3   